In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-10-27 21:20:52.087106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730044252.113927    9678 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730044252.121713    9678 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-27 21:20:52.151545: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
data = pd.read_csv("data/banking-data.csv")
data.head()

,tags,instruction,category,intent,response
0,BCIPZ,"I would like to acivate a card, can you help me?",CARD,activate_card,I'm here to assist you with that! Activating y...
1,BCILZ,"I have to activate an Visa online, how can I d...",CARD,activate_card,I'm here to assist you with activating your {{...
2,BCIPQZ,I'd like to actiate a card where do i do it,CARD,activate_card,I can help you with that! Activating your card...
3,BCLPQZ,I'd likke to activate a visa on mobile i need ...,CARD,activate_card,I'm here to assist you with activating your {{...
4,BCILPZ,"I would ilke to activate a credit card online,...",CARD,activate_card,I'm here to assist you with activating your cr...


In [6]:
# Encode intent labels
label_encoder = LabelEncoder()
data['intent_encoded'] = label_encoder.fit_transform(data['intent'])

# Tokenize instructions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['instruction'])
X = tokenizer.texts_to_sequences(data['instruction'])
X = pad_sequences(X, padding='post')  # Ensure uniform length

# Intent labels as the target variable
y = data['intent_encoded']

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Model architecture
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=X.shape[1]),
    LSTM(64),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=100, validation_split=0.2)


Epoch 1/40


/home/nikhil/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


639/639 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.4159 - loss: 1.8986 - val_accuracy: 0.0759 - val_loss: 15.2657
Epoch 2/40
639/639 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.9509 - loss: 0.1702 - val_accuracy: 0.0783 - val_loss: 15.7038
Epoch 3/40
639/639 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.9859 - loss: 0.0568 - val_accuracy: 0.0795 - val_loss: 17.3168
Epoch 4/40
639/639 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.9933 - loss: 0.0294 - val_accuracy: 0.0795 - val_loss: 17.7799
Epoch 5/40
639/639 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.9918 - loss: 0.0312 - val_accuracy: 0.0793 - val_loss: 17.9606
Epoch 6/40
639/639 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.9975 - loss: 0.0108 - val_accuracy: 0.0797 - val_loss: 19.3512
Epoch 7/40
639/639 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.9966 - loss: 0.0131 - val_accuracy: 0.0791 - val_loss: 18.8116
Epoch 8/40
639/639 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.9986 - loss: 0.0074 - val_

In [30]:
import numpy as np

def get_response(user_input):
    # Process input
    seq = tokenizer.texts_to_sequences([user_input])
    padded_seq = pad_sequences(seq, maxlen=X.shape[1], padding='post')
    
    # Predict intent
    prediction = model.predict(padded_seq)
    intent_idx = np.argmax(prediction)
    intent = label_encoder.inverse_transform([intent_idx])[0]

    # Debug prints
    print(f"User Input: {user_input}")
    print(f"Predicted Intent Index: {intent_idx}, Predicted Intent: {intent}")

    # Fetch response for intent
    if intent in data['intent'].values:
        response = data[data['intent'] == intent]['response'].values[0]
    else:
        response = "I'm not sure how to respond to that."
    
    return response


In [37]:
print(get_response("i want change my password"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
User Input: i want change my password
Predicted Intent Index: 5, Predicted Intent: cancel_card
I'm here to assist you with canceling your {{Credit Card}} on mobile. Let's get it sorted out. 

To cancel your {{Credit Card}} on mobile, please follow these steps:

1. Open your banking app on your mobile device.
2. Log in to your account using your credentials (username and password).
3. Navigate to the "Cards" section or a similar option within the app.
4. Look for the {{Credit Card}} you want to cancel and select it.
5. Find the "Cancel" or "Close" option associated with the selected card.
6. Confirm your decision to cancel the card.
7. Follow any additional prompts or instructions provided by the app to complete the cancellation process.

If you encounter any difficulties or have any further questions, please feel free to reach out to {{Customer Support Phone Number}}. Our dedicated customer support team will be more than happy to assist you further

In [44]:
test_inputs = [
    "I want to activate my card",
    "How do I check my balance?",
    "Can you help me with my password?"
]
for input_text in test_inputs:
    print(get_response(input_text))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
User Input: I want to activate my card
Predicted Intent Index: 0, Predicted Intent: activate_card
I'm here to assist you with that! Activating your card is an important step to starting and enjoying its benefits. Here's how you can activate your card:

1. Locate the activation instructions: Depending on the card issuer, you may find the activation instructions on a sticker attached to the card itself, in the welcome package, or on the issuer's website.

2. Visit the card issuer's activation website: Using your computer or mobile device, open a web browser and navigate to the card issuer's website. Look for the activation page or section.

3. Enter the required information: Follow the prompts on the activation page and provide the necessary information. This may include your card number, personal details, and security code.

4. Set up your card: Once you've entered the required information, you may have the option to set up a PIN, create an online a

In [43]:
test_inputs = [
    "I am trying to ifnd a password, I need help",
    "I want to activate my card",
    "How do I check my balance?"
]
for input_text in test_inputs:
    print(get_response(input_text))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
User Input: I am trying to ifnd a password, I need help
Predicted Intent Index: 3, Predicted Intent: apply_for_mortgage
I can assist you with that. Applying for a mortgage can seem overwhelming, but don't worry, I'm here to help you through the process. Here's what you need to do:

1. Start by researching different mortgage lenders and comparing their rates and terms to find the best fit for you.
2. Gather all the necessary documents, such as proof of income, bank statements, and identification.
3. Contact the selected mortgage lender directly or visit their website to begin the application process.
4. Fill out the application form with accurate and up-to-date information.
5. Submit the required documents along with the application.
6. Once submitted, the lender will review your application and may request additional information if needed.
7. After the review process, the lender will notify you of their decision and provide details on the next step